## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [9]:
from langchain.agents import create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent
import os 
load_dotenv()

True

In [6]:
#llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-002", 
                              google_api_key=os.getenv("GOOGLE_API_KEY"),
                                temperature=0)

In [7]:
tools = load_tools(["llm-math"],  llm=llm)

In [19]:
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferWindowMemory
#memory = ConversationBufferMemory(memory_key="chat_history")
#memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history")
memory = ConversationBufferWindowMemory(memory_key="chat_history")


/var/folders/9m/qp7n28sd3d13_ywgjkkwnbb00000gn/T/ipykernel_67170/3656458992.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(memory_key="chat_history")


In [20]:


conversational_agent = initialize_agent(
    agent="conversational-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

In [21]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
output_2=conversational_agent.run("Add 5 to the previous result")



> Entering new AgentExecutor chain...
```tool_code
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 7 + 9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.
```


> Finished chain.


> Entering new AgentExecutor chain...
```tool_code
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 16 + 5
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 5 to the previous result (16) is 21.
```


> Finished chain.


In [22]:
print(output_1)
print(output_2)

The result of adding 7 to 9 is 16.
```
The result of adding 5 to the previous result (16) is 21.
```


## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [23]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [34]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-002", 
                              google_api_key=os.getenv("GOOGLE_API_KEY"),
                                temperature=0)

#intializing the agent
docstore_agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=20
)

In [35]:
docstore_agent("Tell me a few key things about Mahatma Gandhi.")



> Entering new AgentExecutor chain...
Thought: I need to search for information about Mahatma Gandhi.
Action: Search [Mahatma Gandhi]

Observation: Search  is not a valid tool, try one of [Search, Lookup].
Thought:Action: Search[Mahatma Gandhi]

Observation: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (Sanskrit: महात्मा, meaning great-souled/venerable), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [37]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [38]:
print(agent.invoke("Question: Who was the president of USA when the first Moom landing took place?"))



> Entering new AgentExecutor chain...
Yes.
Follow up: When did the first moon landing take place?
Intermediate answer: Witness Neil Armstrong take humankind's first steps on the lunar surface during NASA's Apollo 11 mission Perhaps the most famous of all space films, these clips document the arrival of the first human beings on the Moon during the afternoon of July 20, 1969. They comprise footage of the landing of the Apollo 11 spacecraft, taken with a 16-mm camera mounted in Edwin Aldrin's window, and Neil ... Watch highlights from the Apollo 11 mission including the launch on July 16, 1969, the landing of the lunar module, Neil Armstrong's first steps on the Moon, splashdown, and more. Watch on NASA+ 20 July 1969—Astronaut Edwin E. Aldrin Jr., lunar module pilot, walks on the surface of the moon near the leg of the Lunar Module (LM) "Eagle ... On 20 July 1969, the world watched in anticipation as Apollo 11's 'Eagle' lunar module touched down on the moon's surface. From the early Ap

The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.